In [3]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

open_charge_map_api_key = os.getenv('OPEN_CHARGE_MAP_API_KEY')
open_charge_map_base_url = 'https://api.openchargemap.io/v3/poi'

params = {
    'output': 'json',
    'countrycode': 'US',
    'latitude': 37.7749,
    'longitude': -122.4149,
    'distance': 50,
    'distanceunit': 'KM',
    'maxresults': 1000,
    'compact': False,   # get detailed data
    'verbose': False,
    'key': open_charge_map_api_key
}

In [5]:
response = requests.get(open_charge_map_base_url, params=params)

if response.status_code == 200:
    data = response.json()
    print(f"Retrieved {len(data)} charging stations.")
    # print(data)
else:
    print(f"Error: {response.status_code}")

Retrieved 1000 charging stations.


In [7]:
# flatten data
df = pd.json_normalize(data)

columns = [
    'ID',
    'AddressInfo.Title',
    'AddressInfo.AddressLine1',
    'AddressInfo.Town',
    'AddressInfo.StateOrProvince',
    'AddressInfo.Postcode',
    'AddressInfo.Country.ISOCode',
    'AddressInfo.Latitude',
    'AddressInfo.Longitude',
    'NumberOfPoints',
    'StatusType.IsOperational',
    'Connections'
]

df = df[columns]

df

,ID,AddressInfo.Title,AddressInfo.AddressLine1,AddressInfo.Town,AddressInfo.StateOrProvince,AddressInfo.Postcode,AddressInfo.Country.ISOCode,AddressInfo.Latitude,AddressInfo.Longitude,NumberOfPoints,StatusType.IsOperational,Connections
0,244541,MONOGRAM RES OLUME 2,1401 Mission St,San Francisco,CA,94103,US,37.774966,-122.416038,NaN,True,"[{'ID': 410594, 'ConnectionTypeID': 1, 'Connec..."
1,29744,169 11th St,169 11th St,San Francisco,California,94103,US,37.773568,-122.415794,NaN,NaN,"[{'ID': 32729, 'ConnectionTypeID': 9, 'Connect..."
2,244538,HUDSON PACIFIC 1455 STATION 10,1455 Market Street,San Francisco,CA,94103,US,37.774846,-122.417113,NaN,True,"[{'ID': 410591, 'ConnectionTypeID': 1, 'Connec..."
3,244540,HUDSON PACIFIC 1455 STATION 1,1455 Market Street,San Francisco,CA,94103,US,37.774933,-122.417260,NaN,True,"[{'ID': 410593, 'ConnectionTypeID': 1, 'Connec..."
4,140926,NEMA Apartments,1411 Market Street,San Francisco,CA,94103,US,37.775819,-122.417239,NaN,True,"[{'ID': 196702, 'ConnectionTypeID': 1, 'Connec..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,262628,ORACLE RWSHORE 5OP-001,520 Oracle Pkwy,Redwood City,CA,94065,US,37.531328,-122.262309,NaN,True,"[{'ID': 445356, 'ConnectionTypeID': 1, 'Connec..."
996,140823,Britannia Point Eden Business Park on Research...,26103 Research Road,Hayward,CA,94545,US,37.625021,-122.127497,NaN,True,"[{'ID': 196564, 'ConnectionTypeID': 1, 'Connec..."
997,27519,CITY OF REDWOOD,399 Marine Pkwy,Redwood City,CA,94065,US,37.531724,-122.259644,NaN,NaN,"[{'ID': 30166, 'ConnectionTypeID': 1, 'Connect..."
998,123978,ORACLE RWSHORE,520 Oracle Pkwy,Redwood City,CA,94065,US,37.530788,-122.261332,NaN,NaN,"[{'ID': 174019, 'ConnectionTypeID': 1, 'Connec..."


In [ ]:
# expand Connections field which contains list of dictionaries with details about each charger
def expand_connections(row):
    connections = row['Connections']
    connection_data = []
    for conn in connections:
        connection_data.append({
            'Station_ID': row['ID'],
            'ConnectionType': conn.get('ConnectionType', {}).get('Title'),
            'Level':
        })